# Anomaly Detection With Clustering

Many data sets take the form of vector values.
One relatively simple but very effective way of defining expected behavior for this kind of data is to cluster it, and use these clusters as a model of our expectations.
Data samples that do not fall inside, or near, any cluster are often anomalous in some way.

In this notebook we will work with
[Amazon Fine Food Reviews](https://www.kaggle.com/snap/amazon-fine-food-reviews/)
data[1] from Kaggle.
We will generate feature vectors from this data and _cluster_ them to use as an anomaly detection model.

[1] J. McAuley and J. Leskovec. From amateurs to connoisseurs: modeling the evolution of user expertise through online reviews. WWW, 2013.

Parquet files used in this notebook were created from the raw Kaggle CSV as follows:
```python
with open("data/amazon-reviews.csv") as f:
    data = pd.read_csv(f)
data = data.sample(10000).reset_index(drop=True)
data = data.drop(columns=["Id", "ProductId", "UserId", "ProfileName", "Time", "Summary"])
data["hscore"] = \
    data.apply(lambda row: (1+row["HelpfulnessNumerator"]) / (2+row["HelpfulnessDenominator"]), axis=1)
data = data.drop(columns=["HelpfulnessNumerator", "HelpfulnessDenominator"])
data = data.rename(columns={"Score":"score", "Text":"text"})
data = data[["score", "hscore", "text"]]
data.to_parquet("data/amazon-reviews-10K.parquet", compression="brotli")
```

In [ ]:
!pip install vaderSentiment altair vega pyarrow

In [ ]:
import codecs
import random
import math
import numpy as np
import scipy
import scipy.stats
from scipy.stats import gamma, kstest
import pandas as pd
import vaderSentiment.vaderSentiment as vader
from sklearn.cluster import KMeans
import re

In [ ]:
import altair as alt
from detail.altairdf import altairDF
alt.renderers.enable("notebook")

In [ ]:
def filterdf(df, pred):
    return df.loc[[idx for idx in df.index if pred(df.loc[idx])]]
def showtxt(df, subset = ["text"]):
    return df.style \
             .applymap(lambda x: 'white-space:wrap', subset=subset) \
             .applymap(lambda x:'text-align:left', subset=subset)

# Loading the data

The raw food review data has been sub-sampled to 50,000 records and stored as a parquet file to reduce its footprint on disk.

We begin by loading the data.
You can see that each review comes with a score, from one to five "stars", a helpfulness score, and the review text.
In this lab we will not be using the helpfulness score.

In [ ]:
reviews = pd.read_parquet("data/amazon-reviews-50K.parquet").reindex()
showtxt(reviews.head(5))

# Sentiment Analysis

Each reviewer supplied a rating from 1 to 5 stars.
One interesting question we might ask is:
does a natural language processing method like sentiment analysis of the reviews themselves agree with this rating?

In this lab we'll use the
[vader sentiment analysis library](https://github.com/cjhutto/vaderSentiment).
The following cell defines a function that breaks each review into sentences and extracts vader's numeric assessment of positive or negative sentiment.
The most positive sentiment is 1.0 and the most negative sentiment is -1.0

In [ ]:
#english = spacy.load('en_core_web_sm')
#def sentences(text):
#    return [str(s) for s in english(text).sents]

sdelim = re.compile('(?<=[.!?]) *')
def sentences(text):
    return [s for s in re.split(sdelim, text) if len(s) > 1]

sentiment = vader.SentimentIntensityAnalyzer()

def sentiment_compound(text):
    scores = [sentiment.polarity_scores(s)['compound'] for s in sentences(text)]
    if len(scores) < 1: return 0.0
    return sum(scores) / len(scores)

In [ ]:
%%time
reviews["sentiment"] = reviews["text"].apply(sentiment_compound)

# A Feature Vector

We'll begin by assembling a simple feature vector, with only two dimensions: the review score, and a corresponding sentiment value taken from vader.
Many clustering algorithms perform best when individual feature dimensions occupy roughly the same scale, and so we will _normalize_ the review scores to be from 0.0 to 1.0 instead of 1 to 5.
In the following we assemble these "2-vectors" as numpy arrays:

In [ ]:
%%time
feats1 = reviews.copy().reindex()
feats1["feats"] = feats1.apply(lambda row: np.array([row["score"] / 5.0, row["sentiment"]]), axis=1)
feats1["feats"].sample(5)

# Visualizing the Features

It is useful to _visualize_ your data.
Since our features have only two dimensions (review score and sentiment), we can directly view our points in these dimensions with a scatter plot.
The following plot illustrates that the review scores exhibit only five values but the corresponding sentiments have much more variation.

In [ ]:
feats1["x: score"] = feats1["feats"].apply(lambda x: x[0])
feats1["y: sentiment"] = feats1["feats"].apply(lambda x: x[1])
alt.Chart(feats1.sample(2000)).encode(x="x: score", y="y: sentiment", color="score").mark_point().interactive()

# Clustering

To build our model of expected food reviews, we will apply k-means clustering.
K-Means clustering is one of the oldest clustering methods.
It is relatively fast, and can be parallelized either via hardware accelerators or scale-out platforms such as Dask or Spark.

In the following cells we apply sklearn's k-means clustering, and overlay the clustering on a scatter plot of our data.

In [ ]:
%%time
data = np.array(list(feats1["feats"]))
clustering = KMeans(n_clusters=10).fit(data)

In [ ]:
feats1["pred"] = clustering.predict(np.array(list(feats1["feats"])))

In [ ]:
feats1["pstr"] = feats1["pred"].apply(str)
alt.Chart(feats1.sample(2000)).encode(x="x: score", y="y: sentiment", color="pstr").mark_point().interactive()

From the scatterplot above, you can see that the clusters do not necessarily align with how a human might place them.
We will see that, at least for anomaly detection, our results are not very sensitive to the particular locations of our clusters.

# Anomalies

In the following cells, we use our clustering to identify some anomalies in our data.
For each review, we compute its distance to the nearest cluster.
Reviews that are not near any cluster are candidate anomalies.
We will sort them so that largest distances are at the top, and look at the first few reviews.

In [ ]:
feats1["pdist"] = feats1.apply(lambda row: np.linalg.norm(row["feats"] - clustering.cluster_centers_[row["pred"]]), axis=1)
feats1["pdist"].sample(5)

In [ ]:
anomalies = feats1.sort_values(by=["pdist"], ascending=False)[["pdist","sentiment","score","text"]].head(25)
showtxt(anomalies)

# Exercises

1. In the anomaly results above, there are cases of a positive review (4 or 5) but negative sentiment. How would you interpret this kind of anomaly?
1. You may also see examples of negative review (1 or 2) but positive sentiment. What would you conclude from this?
1. Some samples may show total agreement between sentiment and anomaly. What is a possible explanation of these  "false positive" anomalies?
1. Can you think of a different way to detect these score/sentiment anomalies that does not require clustering?

# Anomalies from Text

In the previous section we looked at anomalies by clustering two numeric features.
Next we will look at a method for clustering based on the review text, using basic concepts from natural language processing (NLP).

In the following cell, we define some functions for extracting shingles (aka
[n-grams](https://en.wikipedia.org/wiki/N-gram)),
along with some simple logic for cleaning the raw text up.

This simple example illustrates 3-shingles from some text:
```
shingles("dog tail", 3)
=>
"dog"
 "og "
  "g t"
   " ta"
    "tai"
     "ail"
```

The number of possible shingles is quite large, particularly for shingles of size 3 or higher.
To work with this, we will _hash_ our shingles into a fixed-length feature vector that contains the count of each shingle in the review text.
We'll also normalize these vectors so that they are all of length 1, to compensate for texts of differing lengths.

In [ ]:
def shingles(k):
    def kshingles(doc):
        return [doc[i:i + k] for i in range(len(doc) - k + 1)]
    return kshingles

htmlbr = re.compile('<br />')
whitesp = re.compile('\\s+')
def cleantxt(txt):
    clean = re.sub(htmlbr, ' ', txt)
    clean = re.sub(whitesp, ' ', clean)
    clean = clean.lower()
    return clean

def hashing_frequency(vecsize, h, norm = 1.0):
    def hf(words):
        if type(words) is type(""):
            # handle both lists of words and space-delimited strings
            words = words.split(" ")
        hsig = np.zeros(vecsize, dtype=np.float32)
        for term in [w for w in words if len(w) > 0]:
            hsig[h(term) % vecsize] += 1.0
        z = np.linalg.norm(hsig) / norm
        if (z > 0.0): hsig /= z
        return hsig
    return hf

In [ ]:
%%time
sh4 = shingles(4)
hsig = hashing_frequency(512, hash, norm = 1)
feats2 = reviews.copy()
feats2["feats"] = feats2["text"].apply(lambda txt: hsig(sh4(cleantxt(txt))))
feats2["feats"].sample(3)

# Visualizing

As before, we would like to visualize our data.
However, in this case, our shingle-based features have hundreds of dimensions.
So we will apply Principle Component Analysis (PCA) to project our features down to 2 dimensions and observe their structure.

In [ ]:
import sklearn.decomposition

def append_pca_columns(df, featcol, pcacols=["x", "y"]):
    DIMENSIONS = 2
    data = np.array(list(df[featcol]))
    pca2 = sklearn.decomposition.PCA(DIMENSIONS)
    pca = pca2.fit_transform(data)
    pca_df = pd.DataFrame(pca, columns=pcacols)
    df = df.drop(columns=pcacols, errors='ignore')
    df = pd.concat([df, pca_df], axis=1).reindex()
    return df

def pca_features(df, icol, ocol, dimensions=2):
    data = np.array(list(df[icol]))
    pca2 = sklearn.decomposition.PCA(dimensions)
    pca = pca2.fit_transform(data)
    df[ocol] = list(pca)
    return df

In [ ]:
feats2 = append_pca_columns(feats2, "feats")
alt.Chart(feats2.sample(2000)).encode(x="x", y="y", color="score").mark_point().interactive()

# Clustering

As before, we will apply k-means clustering to our new text shingle features.
When we plot the clusters in 2D, there is less separation due to their locations in higher dimensional space being collapsed to 2.
However, we can see some points that look like possible outliers.

In [ ]:
%%time
data = np.array(list(feats2["feats"]))
clustering = KMeans(n_clusters=10).fit(data)

In [ ]:
feats2["pred"] = clustering.predict(np.array(list(feats2["feats"])))
feats2["pstr"] = feats2["pred"].apply(str)
alt.Chart(feats2.sample(2000)).encode(x="x", y="y", color="pstr").mark_point().interactive()

# Anomalies with Text Shingles

Also as before, we can map reviews to their smallest distance to a cluster and sort so the reviews farthest from any cluster are first.

In [ ]:
feats2["pdist"] = feats2.apply(lambda row: np.linalg.norm(row["feats"] - clustering.cluster_centers_[row["pred"]]), axis=1)
feats2["pdist"].sample(5)

In [ ]:
anomalies = feats2.sort_values(by=["pdist"], ascending=False)[["pdist","score","sentiment","text"]].head(25)
showtxt(anomalies)

# Exercises

1. How are the anomalies shown above different than the score/sentiment anomalies?
1. What do these anomalies have in common? How does that relate to features that we collected?

# Anomalies with Word Features

The previous two variations on feature vectors for anomaly detection suggest that there is no one way to define what is "anomalous".
What we detect as an anomaly depends on how we define our expectations.
That in turn depends on what kind of features we collect in the first place.

With that in mind, what will happen if we replace shingles with whole words for generating hashed frequency vectors?

In the following cells we apply the SKLearn hashing vectorizer to create a hashed vector of word counts.
As before, we will normalize these vectors to a length of 1 to put different review lengths on an equal footing.

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer, TfidfTransformer

HVSIZE = 1000
vectorizer = HashingVectorizer(token_pattern='(?u)\\b[A-Za-z]\\w+\\b', n_features = HVSIZE, alternate_sign=False)
hvcounts = vectorizer.fit_transform(reviews["text"].apply(cleantxt))

In [ ]:
def normarray(v):
    r = v.toarray().reshape(HVSIZE)
    z = np.linalg.norm(r)
    if (z > 0.0): r /= z
    return r

feats3 = reviews.copy()
feats3["feats"] = [normarray(v) for v in hvcounts]

# Visualization

Again we use PCA get our feature vectors into a visualizable form.
In this low dimensionality there is relatively little structure evident.

In [ ]:
feats3 = append_pca_columns(feats3, "feats")
alt.Chart(feats3.sample(2000)).encode(x="x", y="y", color="score").mark_point().interactive()

# Clustering Hashed Word Counts

As with hashed shingles, the word-based clusters show a lot of overlap in low dimensional projections,
but possible outliers are present.

In [ ]:
%%time
data = np.array(list(feats3["feats"]))
clustering = KMeans(n_clusters=10).fit(data)

In [ ]:
feats3["pred"] = clustering.predict(np.array(list(feats3["feats"])))
feats3["pstr"] = feats3["pred"].apply(str)
alt.Chart(feats3.sample(2000)).encode(x="x", y="y", color="pstr").mark_point().interactive()

# Hashed Word Anomalies

We apply our now-familiar technique of identifying reviews which are not near any cluster, and sorting by distance.
You can see that word-based features identify different anomalies than shingle-based features, even though they are both representations of the review text.

In [ ]:
feats3["pdist"] = feats3.apply(lambda row: np.linalg.norm(row["feats"] - clustering.cluster_centers_[row["pred"]]), axis=1)
feats3["pdist"].sample(5)

In [ ]:
anomalies = feats3.sort_values(by=["pdist"], ascending=False)[["pdist","score","sentiment","text"]].head(25)
showtxt(anomalies)

# Exercises

1. How are the anomalies detected with hashed word frequencies different than hashed shingles?
1. Can you think of explainations for this difference?
1. Are the anomalies detected by all the different features in this notebook equally useful?